# <center> Майнор "Интеллектуальный анализ данных" <center>

# <center> Курс "Современные методы машинного обучения" <center>

# <center> Ensemble learning methods: Boosting <center>

## Initial setup

In [52]:
import numpy as np
import pandas as pd

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# стиль seaborn
# style.available выводит все доступные стили
#from matplotlib import style
#style.use('seaborn')

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

## Titanic data

Продолжаем работать с данными по пассажирам Титаника.  
Будем использовать уже модифицированные данные (см `DT_CV_ParamTune.ipynb`)

In [49]:
titanic_data = pd.read_csv('titanic_train_mod.csv')
titanic_data.head(10)

FileNotFoundError: File b'titanic_train_mod.csv' does not exist

In [ ]:
titanic_data.info()

## Алгоритм AdaBoost

Сперва разделим данные на обучающие и тестовые. Возьмем тот же seed, что и с деревьями решений.

In [53]:
from sklearn.model_selection import train_test_split

In [ ]:
Data = titanic_data.drop(['PassengerId', 'Survived'], axis=1)
label = titanic_data['Survived']

In [ ]:
(trainData, 
 testData, 
 train_label, 
 test_label) = train_test_split(Data,
                                label,
                                test_size=0.3, 
                                random_state=56428234)

In [ ]:
print('Size of train set:', trainData.shape)
print('Size of test set:', testData.shape)

In [ ]:
print('Class distribution on train:',
      train_label.value_counts(normalize=True),
      '\nClass distribution on test:', 
      test_label.value_counts(normalize=True),
      sep='\n')

<br>
<br>
Применим AdaBoost с дефолтными параметрами

In [54]:
from sklearn.ensemble import AdaBoostClassifier

In [55]:
ab_model = AdaBoostClassifier()

ab_model.fit(trainData, train_label)

NameError: name 'trainData' is not defined

Оценим качество

In [57]:
from sklearn import metrics
#from plot_cm import plot_confusion_matrix

In [51]:
def get_model_quality(test_label, preds):
    print("Accuracy:",
      round(metrics.accuracy_score(test_label, preds), 5),
     '\nBalanced accuracy:',
     round(metrics.balanced_accuracy_score(test_label, preds), 5))

    print()
    print(metrics.classification_report(test_label, preds))

    #plot_confusion_matrix(cm=metrics.confusion_matrix(test_label, preds),
                          #target_names=['0', '1'],
                          #normalize=True)

In [ ]:
get_model_quality(test_label, ab_model.predict(testData))

Проверим устойчивость решения:

In [ ]:
ab_model = AdaBoostClassifier()
ab_model.fit(trainData, train_label)
get_model_quality(test_label, ab_model.predict(testData))

### Задание  
  
Оцените качество модели на кросс-валидации. Используйте метрику F1-score.  
<br>
<br>

In [58]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
ab_model = AdaBoostClassifier()

In [ ]:
ab_scores = pd.Series(cross_val_score(ab_model, Data, label, cv=rskf, scoring='f1_micro'))

In [ ]:
ab_scores.mean()

In [ ]:
ab_scores.describe()

In [ ]:
ab_scores.plot()

plt.ylabel('f1_micro')
plt.xlabel('iteration')

plt.show()

### Base estimators

Пусть в качестве базового классификатора будет логистическая регрессия

In [ ]:
# Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [ ]:
from sklearn.linear_model import  LogisticRegression

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
lg_model = LogisticRegression()
ab_model = AdaBoostClassifier(base_estimator=lg_model)

In [ ]:
ab_scores = pd.Series(cross_val_score(ab_model, Data, label, cv=rskf, scoring='f1_micro'))

In [ ]:
ab_scores.describe()

Или то же дерево решений, но глубины 2

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
dtree_model = DecisionTreeClassifier(max_depth=2)
ab_model = AdaBoostClassifier(base_estimator=dtree_model)

In [ ]:
ab_scores = pd.Series(cross_val_score(ab_model, Data, label, cv=rskf, scoring='f1_micro'))
ab_scores.describe()

### Задание  
  
Оцените качество классификации для различной (от 1 до 10) глубины дерева решения, используемого в качестве base_estimator.  
<br>
<br>

In [ ]:
scores = {x:[] for x in range(1, 11)}

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

for x in scores:
    
    dtree_model = DecisionTreeClassifier(max_depth=x)
    ab_model = AdaBoostClassifier(base_estimator=dtree_model)
    
    scores[x] = cross_val_score(ab_model, Data, label, cv=rskf, scoring='f1_micro')
    
scores = pd.DataFrame(scores)

In [ ]:
scores.describe()

In [ ]:
scores.boxplot(figsize=(8,5))
plt.ylabel('f1-micro')
plt.show()

### Number of estimators

Мы можем, как и в Random Forest получить доступ к каждому дереву в ансамбле. 

In [ ]:
ab_model = AdaBoostClassifier()
ab_model.fit(trainData, train_label)

In [ ]:
estim_scores = []

for x in ab_model.estimators_:
    estim_scores.append(metrics.f1_score(test_label, 
                                         x.predict(testData)))

estim_scores = pd.Series(estim_scores)

А можем получить предсказания на каждом уровне ансамбля

In [ ]:
x = ab_model.staged_predict(testData)
x

In [ ]:
x[2]

In [ ]:
staged_scores = []

for x in ab_model.staged_predict(testData):
    staged_scores.append(metrics.f1_score(test_label, x))
    
staged_scores = pd.Series(staged_scores)

In [ ]:
estim_scores.plot()

plt.ylabel('f1_micro')
plt.xlabel('iteration')

plt.title('By estimators')
plt.show()

In [ ]:
staged_scores.plot()

plt.ylabel('f1_micro')
plt.xlabel('iteration')

plt.title('By stage')
plt.show()

### Задание  
  
Оцените на 5-fold валидации, как меняется качество модели на обучающей и на тестовой выборках при добавлении каждого дерева. Провизуализируйте результаты.
<br>
<br>

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
f1_train = [] # will be list of lists of scores
acc_train = [] # will be list of lists of scores
f1_test = [] # will be list of lists of scores

skf = StratifiedKFold(n_splits=5, shuffle=True)

for train_index, test_index in skf.split(Data, label):
    
    # train data
    x_train = Data.iloc[train_index] # data
    y_train = label.iloc[train_index] # label
    
    # test data
    x_test = Data.iloc[test_index] # data
    y_test = label.iloc[test_index] # label
    
    # model
    ab_model = AdaBoostClassifier()
    ab_model.fit(x_train, y_train) # fitting
    
    # f1 on train on each stage
    scores = []
    for s in ab_model.staged_predict(x_train):
        scores.append(metrics.f1_score(y_train, s))
    f1_train.append(scores)
    del scores
    
    # f1 on test on each stage
    scores = []
    for s in ab_model.staged_predict(x_test):
        scores.append(metrics.f1_score(y_test, s))
    f1_test.append(scores)
    del scores
    
    # acc on train on each stage
    acc_train.append(list(ab_model.staged_score(x_train, y_train)))
    
    
# convert to DataFrame
f1_train = pd.DataFrame(f1_train).transpose()
acc_train = pd.DataFrame(acc_train).transpose()
f1_test = pd.DataFrame(f1_test).transpose()

При обучении алгоритм оптимизирует не точность, которю мы смотрим, а exponential loss, причем взвешенный. Так что на обучающей выборке мы не видим стабильного увеличения метрики (что f1, что accuarcy), но заметен тренд.  
На графике для тестовой выборки можно заметить переобучение в некоторых случаях. Но в целом там творится безумие из-за неустойчивости данных :(

In [ ]:
f1_train.plot()

plt.xlabel('stage')
plt.ylabel('f1-score')
plt.title('f1 on train')

plt.show()

In [ ]:
acc_train.plot()

plt.xlabel('stage')
plt.ylabel('accuracy')
plt.title('Accuracy on train')

plt.show()

In [ ]:
f1_test.plot()

plt.xlabel('stage')
plt.ylabel('f1-score')
plt.title('f1 on test')

plt.show()

### Hyperparameters tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'base_estimator__criterion' : ['gini', 'entropy'],
    'base_estimator__splitter' :   ['best', 'random'],
    'base_estimator__max_depth': range(1, 6),
    'n_estimators': range(10,101,10),
    'learning_rate': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
dtree_model = DecisionTreeClassifier()
ab_model = AdaBoostClassifier(base_estimator=dtree_model)
clf = GridSearchCV(ab_model, params, cv=rskf, scoring='f1_micro', n_jobs=2)

In [ ]:
%%time
clf.fit(Data, label)

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

### Задание  
  
Оцените на 5-fold валидации, как меняется качество модели (используйте `metrics.zero_one_loss()`) на обучающей и на тестовой выборках при добавлении каждого дерева для различных learning_rate: [0.1, 0.5, 0.9]. Провизуализируйте результаты.  
**NB:** Для бинарной классификации `zero_one_score` равен `1 - accuracy`. Эти метрики можно вычислять явно, как в предыдушем задании, а можно использовать `staged_score()`.  
<br>
<br>

# Bank data. SVN.


In [ ]:
bank_data = pd.read_csv(filepath_or_buffer='bank_data_mod.csv')
bank_data.head(10)

In [ ]:
labels = pd.read_csv(filepath_or_buffer='bank_label_bin.csv', header=None)

In [ ]:
labels.info()

In [ ]:
bank_data.info()

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [74]:
skf = StratifiedKFold(n_splits=5)
clf = LinearSVC(tol=1e-5)

In [75]:
clf.fit(bank_data, labels)

E:\Apps\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Apps\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
     verbose=0)

In [65]:
frac_data = bank_data.sample(frac =.25) 

In [66]:
frac_data.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month__jun,month__mar,month__may,month__nov,month__oct,month__sep,poutcome__failure,poutcome__other,poutcome__success,poutcome__unknown
15566,25,0,15,0,0,21,24,3,-1,0,...,0,0,0,0,0,0,0,0,0,1
292,24,0,423,1,0,5,226,3,-1,0,...,0,0,1,0,0,0,0,0,0,1
39355,38,0,976,1,0,18,32,8,308,3,...,0,0,1,0,0,0,0,0,1,0
24892,37,0,2152,1,0,18,200,1,-1,0,...,0,0,0,1,0,0,0,0,0,1
23707,34,0,553,0,0,28,45,12,-1,0,...,0,0,0,0,0,0,0,0,0,1


In [67]:
frac_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11302 entries, 15566 to 28791
Data columns (total 48 columns):
age                     11302 non-null int64
default                 11302 non-null int64
balance                 11302 non-null int64
housing                 11302 non-null int64
loan                    11302 non-null int64
day                     11302 non-null int64
duration                11302 non-null int64
campaign                11302 non-null int64
pdays                   11302 non-null int64
previous                11302 non-null int64
job__admin.             11302 non-null int64
job__blue-collar        11302 non-null int64
job__entrepreneur       11302 non-null int64
job__housemaid          11302 non-null int64
job__management         11302 non-null int64
job__retired            11302 non-null int64
job__self-employed      11302 non-null int64
job__services           11302 non-null int64
job__student            11302 non-null int64
job__technician         11302 no

In [68]:
frac_labels = labels.loc[frac_data.index.values]

In [69]:
frac_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11302 entries, 15566 to 28791
Data columns (total 1 columns):
0    11302 non-null int64
dtypes: int64(1)
memory usage: 176.6 KB


In [70]:
clf.fit(frac_data, frac_labels)

E:\Apps\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Apps\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [71]:
ab_scores = pd.Series(cross_val_score(clf, frac_data, frac_labels, cv=skf, scoring='f1_micro'))

E:\Apps\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Apps\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
E:\Apps\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Apps\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
E:\Apps\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A c

In [72]:
ab_scores

0    0.880637
1    0.887611
2    0.880531
3    0.904425
4    0.894690
dtype: float64

Сделать с обычным SVC, но с ядром Linear и сравнить